In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


TABLAS PARA NUESTRO DATACLEANING Y PREPROCESSING

## Tabla de VENTAS

* Es la **TABLA DE HECHOS**
* Historico de ventas con:
    * pk_sale: ID de venta
    * cid: ID de cliente
    * month_sale: mes venta
    * product_ID: ID producto
    * net_margin: margen neto estimado generado en la venta

In [58]:
ventas=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/datasets_TFM/sales_20_19_NEW.csv', index_col=0)

In [59]:
ventas.head()

,pk_sale,cid,month_sale,product_ID,net_margin
0,6666,33620,2018-05-01,2335,952.9
1,6667,35063,2018-06-01,2335,1625.2
2,6668,37299,2018-02-01,2335,1279.7
3,6669,39997,2018-02-01,2335,1511.9
4,6670,44012,2018-02-01,2335,1680.3


CAMBIO DE FORMATO DE COLUMNA month_sale PARA QUITAR EL DÍA Y UNIFICAR FORMATOS

In [60]:
# Cambio de la columna a formato string
ventas['month_sale']=ventas['month_sale'].astype(str)

In [61]:
# Borrado de los últimos tres caracteres del campo month_sale
ventas['month_sale']=ventas['month_sale'].str[:-3]

In [62]:
ventas.head()

,pk_sale,cid,month_sale,product_ID,net_margin
0,6666,33620,2018-05,2335,952.9
1,6667,35063,2018-06,2335,1625.2
2,6668,37299,2018-02,2335,1279.7
3,6669,39997,2018-02,2335,1511.9
4,6670,44012,2018-02,2335,1680.3


In [63]:
ventas[ventas['cid']==35063]

,pk_sale,cid,month_sale,product_ID,net_margin
1,6667,35063,2018-06,2335,1625.2
123354,130020,35063,2018-05,4657,71.1


In [64]:
print('Ventas totales: {:,}'.format(ventas.shape[0]))
print('Historico de',ventas.month_sale.min(),'a',ventas.month_sale.max())

Ventas totales: 240,773
Historico de 2018-02 a 2019-05


# Tabla de DICCIONARIO DE PRODUCTOS

Tabla diccionario del ID producto. Incluye:
* Nombres de productos
* Familia de productos

In [65]:
dicc_productos = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/datasets_TFM/product_ID_df.csv',index_col=0)
dicc_productos.sort_values('family_product')

,product_ID,product_desc,family_product
0,4657,em_acount,account
3,2234,payroll,account
4,2235,payroll_account,account
5,8871,emc_account,account
7,2335,short_term_deposit,investment
8,2336,long_term_deposit,investment
9,2673,securities,investment
10,1119,funds,investment
11,9001,loans,loan
12,4033,mortgage,loan


# Tabla de PRODUCTOS

Tabla de productos por cliente y fecha
 Incluye:
* Nombres de productos.
* Clientes y fecha de adquisición.

In [66]:
productos = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/datasets_TFM/products_df.csv', index_col=0)
productos.head()

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,1050611,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,1050612,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
3,1050613,2018-01-28,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4,1050614,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


In [67]:
# Cambiamos los nombres de las columnas en productos para que coincidan con ventas y así poder hacer el join.
productos.rename(columns={'pk_partition':'month_sale', 'pk_cid':'cid'}, inplace=True)

In [68]:
productos.head()

,cid,month_sale,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,1050611,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,1050612,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
3,1050613,2018-01-28,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4,1050614,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


## Tabla ACTIVIDAD COMERCIAL

Tabla de atributos de cliente.
* Incluye variables de actividad comercial: segmento del cliente, nivel de actividad, antiguedad y canal de captacion.
* Es una **TABLA PARTICIONADA**, incluye foto de 17 meses (Ene 2018- May 2019). Cada particion incluye toda la base de clientes.

In [69]:
actividad_com = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/datasets_TFM/commercial_activity_df.csv', index_col=0)
actividad_com.head()

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
0,1375586,2018-01,2018-01,KHL,1.0,02 - PARTICULARES
1,1050611,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
2,1050612,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
3,1050613,2018-01,2015-08,KHD,0.0,03 - UNIVERSITARIO
4,1050614,2018-01,2015-08,KHE,1.0,03 - UNIVERSITARIO


In [70]:
# Cambio de los nombres de las columnas en productos para que coincidan con ventas y así poder hacer el join.
actividad_com.rename(columns={'pk_partition':'month_sale', 'pk_cid':'cid'}, inplace=True)

In [71]:
actividad_com.head()

,cid,month_sale,entry_date,entry_channel,active_customer,segment
0,1375586,2018-01,2018-01,KHL,1.0,02 - PARTICULARES
1,1050611,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
2,1050612,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
3,1050613,2018-01,2015-08,KHD,0.0,03 - UNIVERSITARIO
4,1050614,2018-01,2015-08,KHE,1.0,03 - UNIVERSITARIO


## Tabla datos SOCIODEMOGRAFICOS

Tabla de atributos del cliente.
* Incluye info de CLIENTE: pais, edad, salario
* Particiones 17 meses (Ene 2018- May 2019). Cada particion incluye la foto de de la info del cliente ese mes especifico.
* Incluye algunos nulos (<1% en la mayoria, 33% en salario)

In [72]:
sociodemografica = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/datasets_TFM/sociodemographic_df.csv', index_col=0)
sociodemografica.head()

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,1375586,2018-01,ES,29.0,H,35,N,87218.10
1,1050611,2018-01,ES,13.0,V,23,N,35548.74
2,1050612,2018-01,ES,13.0,V,23,N,122179.11
3,1050613,2018-01,ES,50.0,H,22,N,119775.54
4,1050614,2018-01,ES,50.0,V,23,N,NaN


In [73]:
# Cambio de los nombres de las columnas en sociodemografica para que coincidan con ventas y así poder hacer el join.
sociodemografica.rename(columns={'pk_partition':'month_sale', 'pk_cid':'cid'}, inplace=True)

In [74]:
sociodemografica.head()

,cid,month_sale,country_id,region_code,gender,age,deceased,salary
0,1375586,2018-01,ES,29.0,H,35,N,87218.10
1,1050611,2018-01,ES,13.0,V,23,N,35548.74
2,1050612,2018-01,ES,13.0,V,23,N,122179.11
3,1050613,2018-01,ES,50.0,H,22,N,119775.54
4,1050614,2018-01,ES,50.0,V,23,N,NaN


## Tabla de TENENCIA DE PRODUCTOS

Tabla de atributos del cliente
* Incluye info de TENENCIA de productos contratados.
* **IMPORTANTISIMO: no confundir con tabla de ventas. Que un cliente TENGA un producto en el mes 2018-05 no significa que lo haya comprado ese mes, sino que ya tiene contratado ese producto.**
* Es una **TABLA PARTICIONADA**, incluye foto de 17 meses (Ene 2018- May 2019). Cada particion incluye toda la base de clientes.

In [75]:
# Esta tabla que comentó Alejandro no la tocaremos, esta información también la tenemos en la tabla products
tenencia_productos = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/datasets_TFM/customer_products_df.csv', index_col=0)
tenencia_productos.head()

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,1050611,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,1050612,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
3,1050613,2018-01,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4,1050614,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


# Funcion merge() para cruce de tablas en Python

MERGE1 LEFT JOIN de la tabla ventas con actividad_com por cid y month_sale

In [76]:
ventas.head()

,pk_sale,cid,month_sale,product_ID,net_margin
0,6666,33620,2018-05,2335,952.9
1,6667,35063,2018-06,2335,1625.2
2,6668,37299,2018-02,2335,1279.7
3,6669,39997,2018-02,2335,1511.9
4,6670,44012,2018-02,2335,1680.3


In [77]:
actividad_com.head()

,cid,month_sale,entry_date,entry_channel,active_customer,segment
0,1375586,2018-01,2018-01,KHL,1.0,02 - PARTICULARES
1,1050611,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
2,1050612,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
3,1050613,2018-01,2015-08,KHD,0.0,03 - UNIVERSITARIO
4,1050614,2018-01,2015-08,KHE,1.0,03 - UNIVERSITARIO


In [78]:
# A nuestro join le pondremos la extensión clean.
actividad_com_clean=pd.merge(left=ventas, right=actividad_com, how='left', on=['cid','month_sale'])

In [79]:
actividad_com_clean.head()

,pk_sale,cid,month_sale,product_ID,net_margin,entry_date,entry_channel,active_customer,segment
0,6666,33620,2018-05,2335,952.9,2018-04,KHK,1.0,02 - PARTICULARES
1,6667,35063,2018-06,2335,1625.2,2018-04,KHN,1.0,01 - TOP
2,6668,37299,2018-02,2335,1279.7,2018-01,KHK,1.0,01 - TOP
3,6669,39997,2018-02,2335,1511.9,2018-01,KAT,1.0,02 - PARTICULARES
4,6670,44012,2018-02,2335,1680.3,2018-01,KHL,1.0,02 - PARTICULARES


MERGE2 LEFT JOIN de la tabla ventas con productos por cid y month_sale

EN LA TABLA PRODUCTOS TENEMOS QUE HACER EL MISMO CAMBIO QUE EN LA TABLA VENTAS POR EL FORMATO DE LA FECHA(month_sale)

In [80]:
# Cambio de formato month_sale de la tabla productos a string
productos['month_sale']=productos['month_sale'].astype(str)

In [81]:
# Borrado de los últimos tres caracteres del campo month_sale
productos['month_sale']=productos['month_sale'].str[:-3]

In [82]:
productos.head()

,cid,month_sale,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,1050611,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,1050612,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
3,1050613,2018-01,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4,1050614,2018-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


In [83]:
# A nuestro join le pondremos la extensión clean.
products_clean=pd.merge(left=ventas, right=productos, how='left', on=['cid','month_sale'])

In [84]:
products_clean.head()

,pk_sale,cid,month_sale,product_ID,net_margin,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,6666,33620,2018-05,2335,952.9,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,6667,35063,2018-06,2335,1625.2,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,6668,37299,2018-02,2335,1279.7,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
3,6669,39997,2018-02,2335,1511.9,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4,6670,44012,2018-02,2335,1680.3,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


MERGE3 LEFT JOIN de la tabla ventas con la tabla sociodemográfica por cid y month_sale

In [85]:
sociodemografica.head()

,cid,month_sale,country_id,region_code,gender,age,deceased,salary
0,1375586,2018-01,ES,29.0,H,35,N,87218.10
1,1050611,2018-01,ES,13.0,V,23,N,35548.74
2,1050612,2018-01,ES,13.0,V,23,N,122179.11
3,1050613,2018-01,ES,50.0,H,22,N,119775.54
4,1050614,2018-01,ES,50.0,V,23,N,NaN


In [86]:
# A nuestro join le pondremos la extensión clean.
sociodemo_clean=pd.merge(left=ventas, right=sociodemografica, how='left', on=['cid','month_sale'])

In [87]:
sociodemo_clean.head()

,pk_sale,cid,month_sale,product_ID,net_margin,country_id,region_code,gender,age,deceased,salary
0,6666,33620,2018-05,2335,952.9,ES,28.0,H,50,N,138618.57
1,6667,35063,2018-06,2335,1625.2,ES,28.0,H,62,N,125530.41
2,6668,37299,2018-02,2335,1279.7,ES,30.0,V,54,N,NaN
3,6669,39997,2018-02,2335,1511.9,ES,41.0,V,62,N,201575.01
4,6670,44012,2018-02,2335,1680.3,ES,28.0,V,42,N,97601.04


In [88]:
dicc_productos.head()

,product_ID,product_desc,family_product
0,4657,em_acount,account
1,3819,debit_card,payment_card
2,1364,pension_plan,pension_plan
3,2234,payroll,account
4,2235,payroll_account,account


AHORA MISMO CONTAMOS CON CUATRO TABLAS


*   actividad_com_clean
*   products_clean
*   sociodemo_clean
*   dicc_productos



# NUEVAS TABLAS, DATA CLEANING Y PREPROCESSING.

In [89]:
# Merge de actividad_com_clean y sociodemo_clean , este movimiento lo hacemos para crear nuestra tabla de hechos.
ventas_new= pd.merge(left=actividad_com_clean, right=sociodemo_clean, how='left', on=["pk_sale","cid","month_sale",'product_ID', 'net_margin'])

In [90]:
ventas_new.head(1)

,pk_sale,cid,month_sale,product_ID,net_margin,entry_date,entry_channel,active_customer,segment,country_id,region_code,gender,age,deceased,salary
0,6666,33620,2018-05,2335,952.9,2018-04,KHK,1.0,02 - PARTICULARES,ES,28.0,H,50,N,138618.57


In [91]:
#Quitamos algunas columnas para dejar solo las pk y las fk
ventas_new.drop(["region_code", "salary", "gender", "age", "deceased","active_customer", "segment", "entry_date", "entry_channel"], axis=1, inplace=True)

TABLA DE HECHOS

In [92]:
#Nuestra tabla de hechos.
ventas_new.head()

,pk_sale,cid,month_sale,product_ID,net_margin,country_id
0,6666,33620,2018-05,2335,952.9,ES
1,6667,35063,2018-06,2335,1625.2,ES
2,6668,37299,2018-02,2335,1279.7,ES
3,6669,39997,2018-02,2335,1511.9,ES
4,6670,44012,2018-02,2335,1680.3,ES


In [93]:
ventas_new[ventas_new["month_sale"]==2018-10].head(20)

,pk_sale,cid,month_sale,product_ID,net_margin,country_id


In [94]:
#Ponemos los nombres de nuestras columnas en minúscula.
ventas_new.columns = ventas_new.columns.str.lower()

In [95]:
ventas_new[ventas_new["cid"]==1211362].head()

,pk_sale,cid,month_sale,product_id,net_margin,country_id
188,6854,1211362,2018-03,2335,1312.7,ES
7445,14111,1211362,2018-03,2336,930.7,ES
13267,19933,1211362,2018-04,2312,50.9,ES
21688,28354,1211362,2018-02,2234,55.5,ES


In [97]:
#Comprobamos que no tenemos nulos
#def column_report(df_x):
  # a = pd.DataFrame(df_x.dtypes, columns={'dtype'})
   # b = pd.DataFrame(df_x.isna().sum(), columns={'nulls'})
    #b['%nulls'] = round(100*b['nulls']/ df_x.shape[0],2)
    #c = df_x.describe(include='all').transpose()
    #return a.join(b).join(c).sort_values('dtype')

#column_report(ventas_new)

TABLAS DE DIMENSIONES

In [98]:
#Esta es nuestra primera tabla de dimensiones pero la tenemos que transformar.
products_clean.head()

,pk_sale,cid,month_sale,product_ID,net_margin,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,6666,33620,2018-05,2335,952.9,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,6667,35063,2018-06,2335,1625.2,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,6668,37299,2018-02,2335,1279.7,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
3,6669,39997,2018-02,2335,1511.9,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4,6670,44012,2018-02,2335,1680.3,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


In [99]:
products_clean.shape

(240773, 20)

In [100]:
#Comprobamos que la pk cid no es unica en esta tabla
products_clean.cid.nunique()

152754

In [101]:
print(products_clean.duplicated().sum(), 'filas duplicadas')

0 filas duplicadas


In [102]:
#Con esto visualizamos los duplicados en CID, ¡OJO NO BORRAMOS DUPLICADOS!de momento, por que necesitamos analizar las fechas de tenencia de productos de cada cliente
products_clean[products_clean.cid.duplicated()].sort_values(by="cid").head()

,pk_sale,cid,month_sale,product_ID,net_margin,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
123340,130006,16502,2018-10,4657,64.3,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
15946,22612,17970,2018-07,2234,49.5,0,0,0,0,0,0,0,0,1.0,1.0,1,1,1,0,0
6513,13179,17980,2018-05,2336,1336.6,0,0,0,1,0,1,0,0,0.0,0.0,0,0,0,0,0
84499,91165,21719,2018-05,3819,54.8,0,0,0,0,0,0,0,0,0.0,0.0,1,0,1,0,0
123341,130007,21719,2018-12,4657,60.5,0,0,0,0,0,0,0,0,0.0,0.0,0,0,1,0,1


In [103]:
#Prueba que datos obtenidos no son los correctos de cid en base a month_sale( no es la ultima fecha de tenencia de productos, ni la unica con respecto a un cliente)
products_clean[products_clean["cid"]==1211362].head()

,pk_sale,cid,month_sale,product_ID,net_margin,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
188,6854,1211362,2018-03,2335,1312.7,1,0,0,0,0,1,0,0,0.0,0.0,1,0,1,0,0
7445,14111,1211362,2018-03,2336,930.7,1,0,0,0,0,1,0,0,0.0,0.0,1,0,1,0,0
13267,19933,1211362,2018-04,2312,50.9,1,0,0,0,0,1,0,1,1.0,1.0,1,0,1,0,0
21688,28354,1211362,2018-02,2234,55.5,0,0,0,0,0,0,0,0,1.0,1.0,1,0,1,0,0


In [104]:
#Se debe obtener el month_sale maximo de todas las fechas del cliente, asi tenemos todas las tenencias a dia de hoy
tabla=products_clean[["cid","month_sale"]].groupby("cid", as_index=False).max().sort_values(by="cid")

In [105]:
 # Creamos asi una tabla con los maximos meses de cada cliente, que es donde se tienen todos los productos de cada cliente
 tabla.head()

,cid,month_sale
0,15891,2018-07
1,16203,2019-01
2,16502,2019-01
3,17457,2018-09
4,17970,2019-02


In [106]:
tabla.cid.nunique()

152754

In [107]:
len(tabla.cid.unique()) == tabla.shape[0]

True

In [108]:
#Se hace merge entre Tabla y products_clean para cruzar los datos filtrados de mes maximo en el dataframe products_clean
products_clean= pd.merge(left=tabla, right=products_clean, on=['cid','month_sale'])

In [109]:
products_clean.head()

,cid,month_sale,pk_sale,product_ID,net_margin,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,15891,2018-07,130004,4657,66.4,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,16203,2019-01,130005,4657,96.7,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,16502,2019-01,78841,8871,68.6,0,0,0,0,0,0,0,0,0.0,0.0,0,1,0,0,1
3,17457,2018-09,91164,3819,56.0,0,0,0,0,0,1,0,0,0.0,0.0,0,0,1,0,1
4,17970,2019-02,12022,2673,830.3,0,0,0,0,1,0,0,0,0.0,1.0,1,1,1,0,0


In [110]:
products_clean.cid.nunique()

152754

In [111]:
#se comprueba que la pk de esta tabla no es unica
products_clean.shape

(176582, 20)

In [112]:
print('Filas duplicadas: ',products_clean.duplicated().sum())

Filas duplicadas:  0


In [113]:
#Con esto podemos ver que algun cliente desde el inicio de su historico sigue con los mismo productos, sigue con la misma foto todo el tiempo.
products_clean[products_clean["cid"]==41264].sort_values(by="cid")

,cid,month_sale,pk_sale,product_ID,net_margin,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
48,41264,2019-04,41366,1364,8696.3,0,0,0,0,0,0,0,0,0.0,1.0,1,0,1,0,0
49,41264,2019-04,60735,2235,81.9,0,0,0,0,0,0,0,0,0.0,1.0,1,0,1,0,0
50,41264,2019-04,91173,3819,54.0,0,0,0,0,0,0,0,0,0.0,1.0,1,0,1,0,0


In [114]:
#Borramos los duplicados en una columna en concreto, se dejan solo las primeras fechas de cada duplicado
products_clean.drop_duplicates(['cid'],keep= "first", inplace=True)

In [115]:
len(products_clean.cid.unique()) == products_clean.shape[0]

True

In [116]:
#Eliminamos algunas columnas que ya tenemos en nuestra tabla de hechos.
products_clean.drop(['pk_sale','product_ID','net_margin'],axis=1,inplace=True)

In [117]:
products_clean.head()

,cid,month_sale,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,15891,2018-07,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,16203,2019-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,16502,2019-01,0,0,0,0,0,0,0,0,0.0,0.0,0,1,0,0,1
3,17457,2018-09,0,0,0,0,0,1,0,0,0.0,0.0,0,0,1,0,1
4,17970,2019-02,0,0,0,0,1,0,0,0,0.0,1.0,1,1,1,0,0


In [118]:
#Comprobamos si esta todo correcto con respecto a un cliente
products_clean[products_clean["cid"]==41264].sort_values(by="cid")

,cid,month_sale,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
48,41264,2019-04,0,0,0,0,0,0,0,0,0.0,1.0,1,0,1,0,0


In [119]:
#Comprobamos si esta todo correcto con respecto a un cliente, para ver si ha funcionado lo de mes maximo por cliente
products_clean[products_clean["cid"]==1211362].sort_values(by="cid")

,cid,month_sale,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
27918,1211362,2018-04,1,0,0,0,0,1,0,1,1.0,1.0,1,0,1,0,0


PRIMERA TABLA DE DIMENSIONES

In [120]:
#PRIMERA TABLA DE DIMENSIONES
products_clean.head()

,cid,month_sale,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,15891,2018-07,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,16203,2019-01,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,16502,2019-01,0,0,0,0,0,0,0,0,0.0,0.0,0,1,0,0,1
3,17457,2018-09,0,0,0,0,0,1,0,0,0.0,0.0,0,0,1,0,1
4,17970,2019-02,0,0,0,0,1,0,0,0,0.0,1.0,1,1,1,0,0


In [122]:
#column_report(products_clean)

In [123]:
#Rellenamos los nulls
products_clean.payroll.fillna(0,inplace=True)
products_clean.pension_plan.fillna(0,inplace=True)

In [124]:
#Reemplazamos floats por int en las columnas payroll y pension plan
products_clean['payroll'] = products_clean['payroll'].astype(int)
products_clean['pension_plan'] = products_clean['pension_plan'].astype(int)

In [125]:
products_clean.head()

,cid,month_sale,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,15891,2018-07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,16203,2019-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,16502,2019-01,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,17457,2018-09,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1
4,17970,2019-02,0,0,0,0,1,0,0,0,0,1,1,1,1,0,0


In [126]:
products_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152754 entries, 0 to 176581
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   cid                 152754 non-null  int64 
 1   month_sale          152754 non-null  object
 2   short_term_deposit  152754 non-null  int64 
 3   loans               152754 non-null  int64 
 4   mortgage            152754 non-null  int64 
 5   funds               152754 non-null  int64 
 6   securities          152754 non-null  int64 
 7   long_term_deposit   152754 non-null  int64 
 8   em_account_pp       152754 non-null  int64 
 9   credit_card         152754 non-null  int64 
 10  payroll             152754 non-null  int64 
 11  pension_plan        152754 non-null  int64 
 12  payroll_account     152754 non-null  int64 
 13  emc_account         152754 non-null  int64 
 14  debit_card          152754 non-null  int64 
 15  em_account_p        152754 non-null  int64 
 16  em

TRANSFORMACION Y DATACLEANING de sociodemo_clean y actividad_com_clean para generar nuestra segunda tabla de dimensiones sociodemo_actividad

In [127]:
# Trabajamos ahora con nuestra tabla sociodemo_clean
sociodemo_clean.head(5)

,pk_sale,cid,month_sale,product_ID,net_margin,country_id,region_code,gender,age,deceased,salary
0,6666,33620,2018-05,2335,952.9,ES,28.0,H,50,N,138618.57
1,6667,35063,2018-06,2335,1625.2,ES,28.0,H,62,N,125530.41
2,6668,37299,2018-02,2335,1279.7,ES,30.0,V,54,N,NaN
3,6669,39997,2018-02,2335,1511.9,ES,41.0,V,62,N,201575.01
4,6670,44012,2018-02,2335,1680.3,ES,28.0,V,42,N,97601.04


In [128]:
#Decidimos quitar estas filas, las tres primeras ya las tenemos en la tabla de hechos y las dos últimas consideramos que no aportan valor a nuestro estudio.
sociodemo_clean.drop(['pk_sale', 'product_ID', 'net_margin', 'region_code', 'salary' ],axis=1,inplace=True)

In [129]:
sociodemo_clean.head()

,cid,month_sale,country_id,gender,age,deceased
0,33620,2018-05,ES,H,50,N
1,35063,2018-06,ES,H,62,N
2,37299,2018-02,ES,V,54,N
3,39997,2018-02,ES,V,62,N
4,44012,2018-02,ES,V,42,N


In [130]:
sociodemo_clean.shape

(240773, 6)

In [131]:
sociodemo_clean.cid.nunique()

152754

In [132]:
len(sociodemo_clean.cid.unique()) == sociodemo_clean.shape[0]

False

In [133]:
#Borramos los en la columna cliente, porque nos interesa dejar a cid como fk y debe ser única.
sociodemo_clean.drop_duplicates(['cid'], keep = 'last', inplace=True)

In [134]:
print('Filas duplicadas: ',sociodemo_clean.duplicated().sum())

Filas duplicadas:  0


In [135]:
sociodemo_clean.shape

(152754, 6)

In [136]:
#Ahora sí se cumple el criterio de único.
len(sociodemo_clean.cid.unique()) == sociodemo_clean.shape[0]

True

In [137]:

sociodemo_clean.head()

,cid,month_sale,country_id,gender,age,deceased
4,44012,2018-02,ES,V,42,N
18,221716,2018-10,ES,H,41,N
22,245679,2018-06,ES,H,66,N
25,272864,2018-06,ES,H,50,N
34,336613,2018-03,ES,V,32,N


In [138]:
#Transformamos a minusculas los nombres de las columnas
sociodemo_clean.columns = sociodemo_clean.columns.str.lower()

In [141]:
#column_report(sociodemo_clean)

In [142]:
#Reemplazamos el nulo de gender por la moda "V"
sociodemo_clean.gender.fillna('V',inplace=True)

In [143]:

actividad_com_clean.sort_values(by="cid").head()

,pk_sale,cid,month_sale,product_ID,net_margin,entry_date,entry_channel,active_customer,segment
123338,130004,15891,2018-07,4657,66.4,2018-07,KAT,1.0,NaN
123339,130005,16203,2019-01,4657,96.7,2018-12,KAT,1.0,02 - PARTICULARES
123340,130006,16502,2018-10,4657,64.3,2018-09,KHN,1.0,02 - PARTICULARES
72175,78841,16502,2019-01,8871,68.6,2018-09,KHN,1.0,02 - PARTICULARES
84498,91164,17457,2018-09,3819,56.0,2017-09,KAT,1.0,02 - PARTICULARES


In [144]:
#Borramos columnas que ya tenemos.
actividad_com_clean.drop(["pk_sale","month_sale", "product_ID", 'net_margin'],axis=1,inplace=True)

In [145]:
actividad_com_clean.head()

,cid,entry_date,entry_channel,active_customer,segment
0,33620,2018-04,KHK,1.0,02 - PARTICULARES
1,35063,2018-04,KHN,1.0,01 - TOP
2,37299,2018-01,KHK,1.0,01 - TOP
3,39997,2018-01,KAT,1.0,02 - PARTICULARES
4,44012,2018-01,KHL,1.0,02 - PARTICULARES


In [146]:
actividad_com_clean.shape

(240773, 5)

In [147]:
actividad_com_clean.cid.nunique()

152754

In [148]:
actividad_com_clean.drop_duplicates(['cid'], inplace=True)

In [149]:
actividad_com_clean.head()

,cid,entry_date,entry_channel,active_customer,segment
0,33620,2018-04,KHK,1.0,02 - PARTICULARES
1,35063,2018-04,KHN,1.0,01 - TOP
2,37299,2018-01,KHK,1.0,01 - TOP
3,39997,2018-01,KAT,1.0,02 - PARTICULARES
4,44012,2018-01,KHL,1.0,02 - PARTICULARES


In [150]:
actividad_com_clean.shape

(152754, 5)

In [151]:
actividad_com_clean.cid.nunique()

152754

In [152]:
#Aquí limpiamos el campo segment y quitamos los números y el guión.
actividad_com_clean['segment'] = actividad_com_clean['segment'].str.replace("01", "")
actividad_com_clean['segment'] = actividad_com_clean['segment'].str.replace("02", "")
actividad_com_clean['segment'] = actividad_com_clean['segment'].str.replace("03", "")
actividad_com_clean['segment'] = actividad_com_clean['segment'].str.replace("-", "")

In [153]:
actividad_com_clean.head()

,cid,entry_date,entry_channel,active_customer,segment
0,33620,2018-04,KHK,1.0,PARTICULARES
1,35063,2018-04,KHN,1.0,TOP
2,37299,2018-01,KHK,1.0,TOP
3,39997,2018-01,KAT,1.0,PARTICULARES
4,44012,2018-01,KHL,1.0,PARTICULARES


In [156]:
#Vemos los nulos que tenemos.
#column_report(actividad_com_clean)

In [157]:
actividad_com_clean.segment.value_counts()

  PARTICULARES     55231
  UNIVERSITARIO    28513
  TOP               5091
Name: segment, dtype: int64

In [158]:
# Reemplazamos nulos por 'Unknown' en  entry_channel y  por la moda en segment, estos representan la tercera parte de los datos 32.77 y 32.92% respectivamente.
actividad_com_clean.entry_channel.fillna('UNKNOW',inplace=True)
actividad_com_clean.segment.fillna('PARTICULARES',inplace=True)

In [159]:
actividad_com_clean.tail()

,cid,entry_date,entry_channel,active_customer,segment
240768,1553456,2019-05,UNKNOW,1.0,PARTICULARES
240769,1553541,2019-05,UNKNOW,1.0,PARTICULARES
240770,1553559,2019-05,UNKNOW,1.0,PARTICULARES
240771,1553565,2019-05,UNKNOW,0.0,PARTICULARES
240772,1553571,2019-05,UNKNOW,1.0,PARTICULARES


In [160]:
actividad_com_clean['segment'] = actividad_com_clean['segment'].str.strip()

In [161]:
#Reemplazamos floats por int en la columna active_customer
actividad_com_clean['active_customer'] = actividad_com_clean['active_customer'].astype(int)

In [164]:
#Comprobamos carencia de nulos en nuestra tabla.
#column_report(actividad_com_clean)

In [165]:
actividad_com_clean.head()

,cid,entry_date,entry_channel,active_customer,segment
0,33620,2018-04,KHK,1,PARTICULARES
1,35063,2018-04,KHN,1,TOP
2,37299,2018-01,KHK,1,TOP
3,39997,2018-01,KAT,1,PARTICULARES
4,44012,2018-01,KHL,1,PARTICULARES


Juntamos actividad_com_clean con sociodemo, ya que comparten la PK CID

In [166]:
actividad_com_clean.head(1)

,cid,entry_date,entry_channel,active_customer,segment
0,33620,2018-04,KHK,1,PARTICULARES


In [167]:
sociodemo_clean.head(1)

,cid,month_sale,country_id,gender,age,deceased
4,44012,2018-02,ES,V,42,N


SEGUNDA TABLA DE DIMENSIONES

In [168]:
# Obtenemos nueva tabla uniendo sociodemo_clean y actividad_com_clean, esta será nuestra segunda tabla de dimensiones.
sociodemo_actividad= pd.merge(left=sociodemo_clean, right=actividad_com_clean,how='left', on=['cid'])

In [169]:
sociodemo_actividad.head()

,cid,month_sale,country_id,gender,age,deceased,entry_date,entry_channel,active_customer,segment
0,44012,2018-02,ES,V,42,N,2018-01,KHL,1,PARTICULARES
1,221716,2018-10,ES,H,41,N,2018-10,KAS,1,PARTICULARES
2,245679,2018-06,ES,H,66,N,2018-06,KFA,1,PARTICULARES
3,272864,2018-06,ES,H,50,N,2018-05,KAT,1,TOP
4,336613,2018-03,ES,V,32,N,2018-03,KHK,0,PARTICULARES


In [170]:
sociodemo_actividad.drop(['country_id'],axis=1,inplace=True)

In [171]:
sociodemo_actividad.shape

(152754, 9)

In [172]:
sociodemo_actividad.cid.nunique()

152754

In [174]:
#column_report(sociodemo_actividad)

TERCERA TABLA DE DIMENSIONES

In [175]:
# Tercera tabla de dimensiones
dicc_productos.head(15)

,product_ID,product_desc,family_product
0,4657,em_acount,account
1,3819,debit_card,payment_card
2,1364,pension_plan,pension_plan
3,2234,payroll,account
4,2235,payroll_account,account
5,8871,emc_account,account
6,2312,credit_card,payment_card
7,2335,short_term_deposit,investment
8,2336,long_term_deposit,investment
9,2673,securities,investment


In [176]:
dicc_productos.columns = dicc_productos.columns.str.lower()

In [178]:
#column_report(dicc_productos)

Pasamos la tablas CSV

In [179]:
ventas_new.to_csv('ventas_new.csv', index=False)
products_clean.to_csv('products_clean.csv', index=False)
sociodemo_actividad.to_csv('sociodemo_actividad.csv', index=False)
dicc_productos.to_csv('dicc_productos.csv', index=False)

In [180]:
from google.colab import files
files.download('ventas_new.csv')
files.download('products_clean.csv')
files.download('sociodemo_actividad.csv')
files.download('dicc_productos.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

A PARTIR DE AQUÍ YA TENEMOS LAS TABLAS PREPARADAS PARA PASARLAS A POWER BI.

In [181]:
ventas[ventas["cid"]==1400698].sort_values(by="cid")

,pk_sale,cid,month_sale,product_ID,net_margin
29783,36449,1400698,2018-08,2234,79.5
48904,55570,1400698,2018-08,1364,9999.0
66442,73108,1400698,2018-08,2235,109.9
110635,117301,1400698,2018-08,3819,60.4
147701,154367,1400698,2018-06,4657,42.9
